# Tidy Data 
## 6.1 Introduction 
In this chapter, we will learn a consistent way to organize your data in Python using the principle known as tidy data. Once you have tidy data, you will spend much less time munging data from one representation to another, allowing you to spend more time on teh data question you care about. 

In this chapter, we will first learn the definition of tidy data and see it applied to simple toy dataset. Then, we'll dive into the main tool for tidying data: melting. Melting allows you to change the format of your data without changing any of the values. 

## 6.2 Tidy Data
There are three features that make a dataset tidy:
1. Each variable is a column; each column is a variable
2. Each observation is a row; each row is an observation
3. Each value is a cell; each cell is a single value

Why tidy data? 
1. data visualization: seaborn
2. pandas' vectorized operations

Some features for untidy data:
1. A variable might be spread across multiple columns --> Melt the data.
2. An observation might be scattered across multiple rows --> Unstack or pivot the multiple rows into columns. 

# 6.3. Tools to Make Data Tidy with pandas 
## 6.3.1 Melt 


In [1]:
import pandas as pd

df=pd.DataFrame (
    {
        "first": ["John","Mary"],
        "last": ["Doe","Bo"],
        "job":["Nurse","Economist"],
        "height":[5.5,6.0],
        "weight":[130,150],
    }
)
print ("\n Unmelted: ")
print (df)
print ("\n Melted: ")
df.melt (id_vars=["first","last"], var_name ="quantity", value_vars=["height","weight"])
# id_vars is the columns to kept fixed, value_vars is the column to unpivot (combined), and var_name is the name for the new column. 


 Unmelted: 
  first last        job  height  weight
0  John  Doe      Nurse     5.5     130
1  Mary   Bo  Economist     6.0     150

 Melted: 


,first,last,quantity,value
0,John,Doe,height,5.5
1,Mary,Bo,height,6.0
2,John,Doe,weight,130.0
3,Mary,Bo,weight,150.0


In [2]:
# Let's import a data set from WHO: tuburculosis from the World Health Organisation.

df_tb = pd.read_parquet(
    "https://github.com/aeturrell/python4DS/raw/refs/heads/main/data/who_tb_cases.parquet"
)
df_tb.head()

,country,1999,2000
0,Afghanistan,745.0,2666.0
1,Brazil,37737.0,80488.0
2,China,212258.0,213766.0


In [3]:
# In this example, we have two columns for a single variable: year. Let's melt this this.
df_tb.melt (
    id_vars =["country"],
    var_name ="year",
    value_vars =["1999","2000"],
    value_name ="cases",
)
# Now we have one observation per row.

,country,year,cases
0,Afghanistan,1999,745.0
1,Brazil,1999,37737.0
2,China,1999,212258.0
3,Afghanistan,2000,2666.0
4,Brazil,2000,80488.0
5,China,2000,213766.0


## 6.3.2.A simpler wide to long 

In [7]:

import numpy as np

df = pd.DataFrame({
    "A1970": ["a", "b", "c"],
    "A1980": ["d", "e", "f"],
    "B1970": [2.5, 1.2, 0.7],
    "B1980": [3.2, 1.3, 0.1],
    "X": np.random.randn(3),
    "id": range(3)
})

print(df)


  A1970 A1980  B1970  B1980         X  id
0     a     d    2.5    3.2  0.361449   0
1     b     e    1.2    1.3 -0.535928   1
2     c     f    0.7    0.1 -0.408553   2


In [8]:
pd.wide_to_long (df,stubnames=["A","B"],i="id",j="year")

,,X,A,B
id,year,,,
0,1970,0.361449,a,2.5
1,1970,-0.535928,b,1.2
2,1970,-0.408553,c,0.7
0,1980,0.361449,d,3.2
1,1980,-0.535928,e,1.3
2,1980,-0.408553,f,0.1


## 6.3.3. Stack and Unstack 
Stack is a shortcut for taking a single type of wide data variable from columns and turning it into a long form dataset, but with an extra index. 

In [22]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)
index = pd.MultiIndex.from_tuples(tuples, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df

A         B
first second                    
bar   one     0.496748 -0.259671
      two     0.137329 -0.439484
baz   one    -0.888434  0.112439
      two     0.381474  0.456567
foo   one     0.056421 -1.075813
      two     1.100903 -0.533316
qux   one     0.145115 -0.153631
      two    -1.453998 -0.549574

In [23]:
df= df.stack ()
df
# This created a multi-layered index but can be reverted to a numbered index using df.reset_index()

first  second   
bar    one     A    0.496748
               B   -0.259671
       two     A    0.137329
               B   -0.439484
baz    one     A   -0.888434
               B    0.112439
       two     A    0.381474
               B    0.456567
foo    one     A    0.056421
               B   -1.075813
       two     A    1.100903
               B   -0.533316
qux    one     A    0.145115
               B   -0.153631
       two     A   -1.453998
               B   -0.549574
dtype: float64

In [25]:
df.unstack (level=0) # level=0 means: take the outermost level of the row index and turn it into columns 

first          bar       baz       foo       qux
second                                          
one    A  0.496748 -0.888434  0.056421  0.145115
       B -0.259671  0.112439 -1.075813 -0.153631
two    A  0.137329  0.381474  1.100903 -1.453998
       B -0.439484  0.456567 -0.533316 -0.549574

In [29]:
df.unstack() # The default is to unstack the innermost index 

A         B
first second                    
bar   one     0.496748 -0.259671
      two     0.137329 -0.439484
baz   one    -0.888434  0.112439
      two     0.381474  0.456567
foo   one     0.056421 -1.075813
      two     1.100903 -0.533316
qux   one     0.145115 -0.153631
      two    -1.453998 -0.549574

In [30]:
df.unstack().unstack()

A                   B          
second       one       two       one       two
first                                         
bar     0.496748  0.137329 -0.259671 -0.439484
baz    -0.888434  0.381474  0.112439  0.456567
foo     0.056421  1.100903 -1.075813 -0.533316
qux     0.145115 -1.453998 -0.153631 -0.549574

In [31]:
df.unstack (level=1)

second        one       two
first                      
bar   A  0.496748  0.137329
      B -0.259671 -0.439484
baz   A -0.888434  0.381474
      B  0.112439  0.456567
foo   A  0.056421  1.100903
      B -1.075813 -0.533316
qux   A  0.145115 -1.453998
      B -0.153631 -0.549574

## 6.3.4.Pivoting data from long to wide
pivot () and pivot_table () help you to sort out data in which a single observation is scatterd over multiple rows.

In [32]:
df_tb_cp = pd.read_parquet(
    "https://github.com/aeturrell/python4DS/raw/refs/heads/main/data/who_tb_case_and_pop.parquet"
)
df_tb_cp.head()

,country,year,type,count
0,Afghanistan,1999-01-01,cases,745
1,Afghanistan,1999-01-01,population,19987071
2,Afghanistan,2000-01-01,cases,2666
3,Afghanistan,2000-01-01,population,20595360
4,Brazil,1999-01-01,cases,37737


In [34]:
# For each year-country, "case" and "population" in different rows. Let's pivot this:
pivoted = df_tb_cp.pivot (
    index =["country","year"], columns =["type"],values ="count"
).reset_index ()
pivoted

# That means each unique pair of (country, year) will become one row.
# DataFrame.pivot(index=None, columns=None, values=None)

type,country,year,cases,population
0,Afghanistan,1999-01-01,745,19987071
1,Afghanistan,2000-01-01,2666,20595360
2,Brazil,1999-01-01,37737,172006362
3,Brazil,2000-01-01,80488,174504898
4,China,1999-01-01,212258,1272915272
5,China,2000-01-01,213766,1280428583
